# Code for scraping Yahoo Finance

This file contains two main chunks of code, one for getting sustainability data from Yahoo Finance, and the other for getting industry and employee count from Yahoo Finance. All relevant files are in the GitHub repository.

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime
import csv
import requests
import pandas as pd
import re
# !pip install html5lib
import urllib.request
# !pip install pywebcopy 
# !pip install pyquery
# !pip install w3lib
# !pip install parse 
# !pip install lxml
from pywebcopy import save_webpage
import lxml
from lxml import html
from os import listdir

In [2]:
# This is a list of all the tickers from the SEC 10-K dataset
ticker_list = pd.read_csv("/Users/phoebeliu/Downloads/SEC_Kaggle_Tickers.csv", index_col = False)
ticker_list = ticker_list["0"]
ticker_list[:10]
len(ticker_list)

4588

In [ ]:
# get list of all URLs 
# sample URL from which we're trying to get sustainalytics data
    # https://finance.yahoo.com/quote/AAPL/sustainability?p=AAPL
    # https://finance.yahoo.com/quote/FKYS/sustainability?p=FKYS
url_list = []
for ticker in ticker_list:
    url1 = "https://finance.yahoo.com/quote/"
    url2 = "/sustainability?p="
    url_list.append(f"https://finance.yahoo.com/quote/{ticker}/sustainability?p={ticker}")

# convert to DF because we want to save the file
url_list = pd.DataFrame(url_list, columns = ["URL"])

# add tickers to it 
ticker_df = pd.DataFrame(np.array(ticker_list), columns = ["Ticker"])
url_list = pd.concat([ticker_df, url_list], axis=1)

url_list.to_csv("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooURLList.csv")
ticker_df.to_csv("/Users/phoebeliu/Documents/SDS_Senior_Project/TickerList.csv")
url_list[:10]

# Get/save the HTML files to disk

In [ ]:
import requests
import sys
from multiprocessing import Pool
from SDS_Senior_Project import fleepy
import time
# url_list.to_html("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooURLList.csv")

def gen_rows(df):
    for row in df.itertuples(index=False):
        yield row._asdict()
        
for row in gen_rows(url_list):
    fleepy.foo(row)
    time.sleep(2.5)
    
# with Pool() as p:
#         p.map(fleepy.foo, gen_rows(url_list))

In [ ]:
# Checking to make sure it worked
len(listdir("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooFinance_HTMLs"))

In [ ]:
# opening csv file and setting up fields with BeautifulSoup
csv_file = open("/Users/phoebeliu/Documents/SDS_Senior_Project/yahoo_sustainalytics_score.csv", "w")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['ticker','total ESG score', 'percentile', "e_score", "s_score", "g_score", "controversy"])
all_htmls = listdir("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooFinance_HTMLs")

count = 0
# looping through all the tickers in tickerlist
for ticker_file in all_htmls:
    # all_htmls[1].split('.')[0]
    temp_ticker = ticker_file.split('.')[0]
    with open(r'/Users/phoebeliu/Documents/SDS_Senior_Project/YahooFinance_HTMLs/' + ticker_file, "r") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html")
    datapanel = soup.find("div",{"class": "Pb(20px) Bdbs(s) Bdbw(3px) Bdbc($seperatorColor) smartphone_Pb(10px) smartphone_Px(20px) smartphone_Bdbw(4px) smartphone_Bdbc($seperatorColor)"})
    if datapanel is not None:
        total_score = datapanel.find("div", {"class": "Fz(36px) Fw(600) D(ib) Mend(5px)"}).text.lstrip()
        percentile = datapanel.find("span",{"class": "Bdstarts(s) Bdstartw(0.5px) Pstart(10px) Bdc($seperatorColor) Fz(12px) smartphone_Bd(n) Fw(500)"}).text.lstrip()

        controversy = soup.find("div",{"class": "D(ib) Fz(36px) Fw(500)"})
        if controversy is not None:
            controversy = controversy.text.lstrip()
        else:
            controversy = ""
        components = datapanel.find_all("div",{"class": "D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)"})
        if len(components) > 0:
            env_score = components[0].text.lstrip()
            social_score = components[1].text.lstrip()
            governance_score = components[2].text.lstrip()
        else:
            env_score = ""
            social_score = ""
            governance_score = ""
        
        csv_writer.writerow([temp_ticker, total_score, percentile, env_score, social_score, governance_score, controversy])
        # print(total_score, percentile, env_score, controversy)
    else:
        print(temp_ticker)
        print(count)
        count = count+1
    # else don't write the row in the csv
          
csv_file.close()


# Do the same thing but for the other data (industry/headcount)

In [ ]:
# get list of all URLs 
# sample URL from which we're trying to get sustainalytics data
    # https://finance.yahoo.com/quote/AAPL/profile?p=AAPL
url_list_profiles = []
for ticker in ticker_list:
    url_list_profiles.append(f"https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}")

# convert to DF because we want to save the file
url_list_profiles = pd.DataFrame(url_list_profiles, columns = ["URL"])

# add tickers to it 
ticker_df = pd.DataFrame(np.array(ticker_list), columns = ["Ticker"]) # don't think i need this bc it was from
# the previous thing
url_list_profiles = pd.concat([ticker_df, url_list_profiles], axis=1)
url_list_profiles.to_csv("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooURLList_Profiles.csv")
url_list_profiles[:10]

In [ ]:
import requests
import sys
from multiprocessing import Pool
from SDS_Senior_Project import turt
import time
# url_list.to_html("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooURLList.csv")

def gen_rows(df):
    for row in df.itertuples(index=False):
        yield row._asdict()
        
for row in gen_rows(url_list_profiles):
    turt.foo(row)
    time.sleep(2.5)
    
# with Pool() as p:
#         p.map(fleepy.foo, gen_rows(url_list))

In [ ]:
urllist_p2 = pd.read_csv("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooURLList_Profiles_Sub.csv")
urllist_p2["URL"][:10]

for row in gen_rows(urllist_p2):
    turt.foo(row)
    time.sleep(2.5)

In [27]:
# opening csv file and setting up fields
csv_file_p = open("/Users/phoebeliu/Documents/SDS_Senior_Project/yahoo_company_profile_2.csv", "w")
csv_writer_p = csv.writer(csv_file_p)
csv_writer_p.writerow(['ticker', 'sector', "industry", "employees", "description"])
all_htmls_p = listdir("/Users/phoebeliu/Documents/SDS_Senior_Project/YahooFinance_Profile_HTMLs")

# need to go back through and clean things

count = 0
# looping through all the tickers in tickerlist
for ticker_file in all_htmls_p:
    # all_htmls[1].split('.')[0]
    temp_ticker = ticker_file.split('.')[0]
    with open(r'/Users/phoebeliu/Documents/SDS_Senior_Project/YahooFinance_Profile_HTMLs/' + ticker_file, "r") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html")
    
    # panel 1:
    if soup.find("div",{"class": "Mb(25px)"}) is None:
        # print(temp_ticker)
        continue
    datapanel = soup.find("div",{"class": "Mb(25px)"})

    # print(datapanel)
    # print(datapanel.find("p", attrs={'class' : 'D(ib) W(47.727%) Pend(40px)'})) 
    # address_info = datapanel.find("p", attrs={'class' : 'D(ib) W(47.727%) Pend(40px)'}).get_text(separator='|')
    if datapanel.find("p", attrs={'class' : 'D(ib) Va(t)'}) is not None:
        industry_info = datapanel.find("p", attrs={'class' : 'D(ib) Va(t)'}).get_text(separator='|')
        industry_info = industry_info.split('|')
    else:
        print(temp_ticker)
        continue

    if (len(industry_info) > 2):
        sector = industry_info[2]
    else:
        sector = ""
    if (len(industry_info) >5):
        industry = industry_info[5]
    else:
        industry = ""
    if (len(industry_info) > 8):
        n_employees = industry_info[8]
    else:
        n_employees = ""
        
    if soup.find("p", attrs={'class':"Mt(15px) Lh(1.6)"}) is not None:
        description = soup.find("p", attrs={'class':"Mt(15px) Lh(1.6)"}).text
    else:
        description = ""


    csv_writer_p.writerow([temp_ticker, sector, industry, n_employees, description])
          
csv_file_p.close()

# this worked for like 3000, try again for more
# the ones that don't work
# UDN etc. -- it's a fund (should i look at funds ???? )
# TVTY -- no longer a public company
# AGGX -- just blank
# XSPA -- is not a public company
# CDEV, MNKKQ -- changed its ticker (without the KQ on yahoo finance)
# TLGTQ -- also seems fake

UDN
TVTY
AGGX
XSPA
AVCO
CDEV
TYME
REGI
TLGTQ
MNKKQ
GCC
UNSS
FRLI
SAIL
POLY
UNL
PEYE
BDR
WSFT
ARNA
APTS
PING
SPIN
GNBT
BDRY


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte